In [1]:
import itk
from itk import TubeTK as ttk

from itkwidgets import view

import numpy as np

import matplotlib.pyplot as plt

In [2]:
ImageType = itk.Image[itk.F, 3]

ReaderType = itk.ImageFileReader[ImageType]

reader1 = ReaderType.New(FileName="Data/mra.mha")
reader1.Update()
im1 = reader1.GetOutput()

reader2 = ReaderType.New(FileName="Data/mri_t1_sag.mha")
reader2.Update()
im2 = reader2.GetOutput()

reader3 = ReaderType.New(FileName="Data/mri_t2.mha")
reader3.Update()
im3 = reader3.GetOutput()

In [3]:
RegisterImagesType = ttk.RegisterImages[itk.F, 3]

reg2To1 = RegisterImagesType.New(FixedImage=im1, MovingImage=im2)
reg2To1.SetReportProgress(True)
reg2To1.SetExpectedOffsetMagnitude(5)
reg2To1.SetExpectedRotationMagnitude(0.005)
reg2To1.SetRegistration("PIPELINE_AFFINE")
reg2To1.SetMetric("MATTES_MI_METRIC")
reg2To1.Update()
im2Reg = reg2To1.ResampleImage()

In [4]:
reg3To1 = RegisterImagesType.New(FixedImage=im1, MovingImage=im3)
reg3To1.SetReportProgress(True)
reg3To1.SetExpectedOffsetMagnitude(5)
reg3To1.SetExpectedRotationMagnitude(0.005)
reg3To1.SetRegistration("PIPELINE_AFFINE")
reg3To1.SetMetric("MATTES_MI_METRIC")
reg3To1.Update()
im3Reg = reg3To1.ResampleImage()

In [5]:
CheckerboardImagesType = itk.CheckerBoardImageFilter[ImageType]
cb23 = CheckerboardImagesType.New(Input1=im2Reg, Input2=im3Reg)
cb23.Update()
im23 = ImageType.New()
im23 = cb23.GetOutput()
view(im23)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageF3; proxy o…

In [6]:
readerBase1 = ReaderType.New(FileName="data/Normal026-FLASH.mha")
readerBase1.Update()
imBase1 = readerBase1.GetOutput()
readerBase1 = ReaderType.New(FileName="data/Normal026-FLASH-Brain.mha")
readerBase1.Update()
imBase1B = readerBase1.GetOutput()

readerBase2 = ReaderType.New(FileName="data/Normal045-FLASH.mha")
readerBase2.Update()
imBase2 = readerBase2.GetOutput()
readerBase2 = ReaderType.New(FileName="data/Normal045-FLASH-Brain.mha")
readerBase2.Update()
imBase2B = readerBase2.GetOutput()

readerBase3 = ReaderType.New(FileName="data/Normal063-FLASH.mha")
readerBase3.Update()
imBase3 = readerBase3.GetOutput()
readerBase3 = ReaderType.New(FileName="data/Normal063-FLASH-Brain.mha")
readerBase3.Update()
imBase3B = readerBase3.GetOutput()

readerBase4 = ReaderType.New(FileName="data/Normal071-FLASH.mha")
readerBase4.Update()
imBase4 = readerBase4.GetOutput()
readerBase4 = ReaderType.New(FileName="data/Normal071-FLASH-Brain.mha")
readerBase4.Update()
imBase4B = readerBase4.GetOutput()

imBase = [imBase1, imBase2, imBase3, imBase4]
imBaseB = [imBase1B, imBase2B, imBase3B, imBase4B]

In [7]:
regB = []
regBB = []
for i in range(0,4):
    regBTo1 = RegisterImagesType.New(FixedImage=im2Reg, MovingImage=imBase[i])
    regBTo1.SetReportProgress(True)
    regBTo1.SetExpectedOffsetMagnitude(5)
    regBTo1.SetExpectedRotationMagnitude(0.005)
    regBTo1.SetRegistration("PIPELINE_AFFINE")
    regBTo1.SetMetric("MEAN_SQUARED_ERROR_METRIC")
    regBTo1.Update()
    img = regBTo1.ResampleImage()
    regB.append( img )
    img = regBTo1.ResampleImage("LINEAR", imBaseB[i])
    regBB.append( img )

In [8]:
regBBT = []
for i in range(0,4):
    imMath = ttk.ImageMath[3].New( Input=regBB[i] )
    imMath.Threshold(0,1,0,1)
    img = imMath.GetOutput()
    if i==0:
        imMathSum = ttk.ImageMath[3].New( img )
        imMathSum.AddImages( img, 0.25, 0 )
        sumBBT = imMathSum.GetOutput()
    else:
        imMathSum = ttk.ImageMath[3].New( sumBBT )
        imMathSum.AddImages( img, 1, 0.25 )
        sumBBT = imMathSum.GetOutput()
        
view(sumBBT)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageF3; proxy o…

In [9]:
insideMath = ttk.ImageMath[3].New( Input = sumBBT )
insideMath.Threshold(1,1,1,0)
brainInside = insideMath.GetOutput()

outsideMath = ttk.ImageMath[3].New( Input = sumBBT )
outsideMath.Threshold(0,0,1,0)
outsideMath.Erode(7,1,0)
brainOutsideAll = outsideMath.GetOutput()
outsideMath.Erode(20,1,0)
outsideMath.AddImages(brainOutsideAll, -1, 1)
brainOutside = outsideMath.GetOutput()

outsideMath.AddImages(brainInside,1,2)
brainCombinedMask = outsideMath.GetOutputUChar()

outsideMath.AddImages(im2Reg, 255, 1)
brainCombinedView = outsideMath.GetOutput()
view(brainCombinedMask)

Viewer(geometries=[], gradient_opacity=0.22, point_sets=[], rendered_image=<itkImagePython.itkImageUC3; proxy …

In [10]:
segmenter = ttk.SegmentConnectedComponentsUsingParzenPDFs[itk.F, 3].New()
segmenter.SetFeatureImage( im1 )
segmenter.AddFeatureImage( im2Reg )
segmenter.AddFeatureImage( im3Reg )
segmenter.SetInputLabelMap( brainCombinedMask )
segmenter.SetObjectId( 2 )
segmenter.AddObjectId( 3 )
segmenter.Update()
brainMask = segmenter.GetOutputLabelMap()

view(brainMask)



TypeError: in method 'tubeSegmentConnectedComponentsUsingParzenPDFsF3_AddObjectId', argument 2 of type 'unsigned char &'